In [ ]:
import os
import cv2
import time
import pydicom
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image,ImageDraw

In [ ]:
#dirpath = "C:/Users/i6root/Tkt/New study/dicomimagedatas/CT_1of2_2DForUse_retable"
dirpath = "C:/Users/i6root/Tkt/New study/dicomimagedatas/CT_2of2_2DForUse_retable"
print(os.path.exists(dirpath))

In [ ]:
file_cut = 0 #スライス枚数を変えないとき
#file_cut = 100
image_size = 512

imglist = []
filepathlist = []
for folder,subfolder,files in os.walk(dirpath):
    if subfolder == []:
        tmpimg=[]
        tmpfile=[]
        
        for fn in tqdm(sorted(files)):
            bn,ext=os.path.splitext(fn)
            filepath = os.path.join(folder,fn)
            #DICOM画像の読み込み
            dcm=pydicom.read_file(filepath,force=True)
            img = dcm.pixel_array
            img = np.array(img, dtype ="float32")
            img = cv2.resize(img, (image_size, image_size), interpolation=cv2.INTER_AREA)
            #形式変換
            try:
                rescale=dcm.RescaleIntercept
            except:
                rescale=0
            try:
                windowcenter = dcm.WindowCenter
                windowwidth= dcm.WindowWidth
                amin= windowcenter-(windowwidth/2)-rescale
                amax= windowcenter+(windowwidth/2)-rescale
                if "_MR" in filepath:
                    amin = 0
                    amax = amax-amin
            except TypeError:
                windowcenter = dcm.WindowCenter[0]
                windowwidth= dcm.WindowWidth[0]
                amin= windowcenter-(windowwidth/2)-rescale
                amax= windowcenter+(windowwidth/2)-rescale
                if "_MR" in filepath:
                    amin = 0
                    amax = amax-amin
            scale = 256.0/(amax-amin+1) # 256.0/(amax-amin+1)
            pixelsByte = img
            pixelsByte = pixelsByte - amin # 最小値は差分　20210622修正
            pixelsByte = pixelsByte*scale # 0-255の範囲にリスケール
            #pixelsByte = np.uint8(pixelsByte)# 符号なしバイトに変換
            pixelsByte[pixelsByte > 255] = 255
            pixelsByte[pixelsByte < 0] = 0 
            #画像の保存
            bn,ext=os.path.splitext(filepath)
            savename=bn+".png"
            savename=savename.replace("dicomimagedatas","imagedatas")
            if file_cut != 0 and "CT_1of2" in dirpath:
                savename=savename.replace("CT_1of2","CT_1of2_"+str(file_cut))
            if file_cut != 0 and "CT_2of2" in dirpath:
                savename=savename.replace("CT_2of2","CT_2of2_"+str(file_cut))
            if file_cut != 0 and "MR" in dirpath:
                savename=savename.replace("MR","MR_"+str(file_cut))
            tmpfile.append(savename)
            filepathlist.append(savename)
            rsurasshu_index = savename.rfind('\\')
            os.makedirs(savename[:rsurasshu_index+1],exist_ok=True)
            cv2.imwrite(savename,pixelsByte)
            if len(tmpfile) == file_cut:
                break
        #filepathlist.append(tmpfile)


In [ ]:
len(filepathlist)

In [ ]:
print(filepathlist)

In [ ]:
#タグの仕分け（train,valid,test）
from sklearn.model_selection import train_test_split
filepathlist_0=[]
filepathlist_1=[]
for i in range(len(filepathlist)):
    if "acceptable" in filepathlist[i]:
        filepathlist_0.append(filepathlist[i])
    if "coldspot" in filepathlist[i]:
        filepathlist_1.append(filepathlist[i])

In [ ]:
try:
    train_0,test_0 = train_test_split(filepathlist_0,train_size=0.7)
    valid_0,test_0 = train_test_split(test_0,train_size=0.5)
    print(len(train_0))
    print(len(test_0))
    print(len(valid_0))
except ValueError:
    print("empty")

In [ ]:
try:
    train_1,test_1 = train_test_split(filepathlist_1,train_size=0.7)
    valid_1,test_1 = train_test_split(test_1,train_size=0.5)
    print(len(train_1))
    print(len(test_1))
    print(len(valid_1))
except ValueError:
    print("empty")

In [ ]:
if filepathlist_0!=[] and filepathlist_1!=[]:
    train = train_0 + train_1 
    valid = valid_0 + valid_1
    test = test_0 + test_1
if filepathlist_1==[]:
    train= train_0
    valid= valid_0
    test = test_0
if filepathlist_0==[]:
    train= train_1
    valid= valid_1
    test = test_1
print(len(train),len(valid),len(test))
print(train)

In [ ]:
# 初めにトレインからパスを格納
temp_path = []
temp_path_tag = []

for i in range(len(train)):
    temp_path.append(train[i])
    temp_path_tag.append("train")
        
for i in range(len(valid)):
    temp_path.append(valid[i])
    temp_path_tag.append("valid")

for i in range(len(test)):
    temp_path.append(test[i])
    temp_path_tag.append("test")
        
import csv
        # パスを保存
if "CT_1of2" in dirpath:
    if file_cut == 0:
        with open('file_way_CT_1of2_2DForUse_retable.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})
    if file_cut != 0:
        with open('file_way_1of2_2DForUse_retable_'+str(file_cut)+'.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})            
if "CT_2of2" in dirpath:
    if file_cut == 0:
        with open('file_way_CT_2of2_2DForUse_retable.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})
    if file_cut != 0:
        with open('file_way_CT_2of2_2DForUse_retable_'+str(file_cut)+'.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})            
if "_MR" in dirpath:
    if file_cut == 0:
        with open('file_way_MR.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})
    if file_cut != 0:
        with open('file_way_MR_'+str(file_cut)+'.csv','w') as csv_file:
            fieldnames = ['file','tag']
            writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
            writer.writeheader()
            for n in range(len(temp_path)):
                writer.writerow({'file':temp_path[n],'tag':temp_path_tag[n]})                